# Decorators & Function Behavior

# What Is a Decorator?

A decorator in Python is a function that takes another function (or class) as an argument and returns a new function that enhances or modifies the behavior of the original.

# It’s often used for:

Logging

Authentication

Timing

Caching

Validation

Pre/post-processing logic

In [2]:
def decorator_function(original_function):
    def wrapper_function(*args, **kwargs):
        print(f"Before calling {original_function.__name__}")
        result = original_function(*args, **kwargs)
        print(f"After calling {original_function.__name__}")
        return result
    return wrapper_function

@decorator_function
def greet(name):
    print(f"Hello, {name}!")


In [3]:
greet("Pritam")

Before calling greet
Hello, Pritam!
After calling greet


# Step-by-Step Breakdown

### Step 1: Define the Decorator
def decorator_function(original_function):

This is a decorator function.

It takes another function as an argument — here it will take greet.

## Step 2: Define the Wrapper Inside It

    def wrapper_function(*args, **kwargs):
        print(f"Before calling {original_function.__name__}")
        result = original_function(*args, **kwargs)
        print(f"After calling {original_function.__name__}")
        return result


This is a nested function that wraps the original one.

It :

Prints something before calling the original function.

Calls the original function with its arguments.

Prints something after.

Returns whatever the original function returns.

## Step 3: Return the Wrapper

return wrapper_function

The outer function returns this wrapper, not the original function itself.

## Step 4: Decorate greet

In [6]:
@decorator_function
def greet(name):
    print(f"Hello, {name}!")

This is syntactic sugar for:

greet = decorator_function(greet)


So the original greet() is replaced by wrapper_function().


# Step 5: Call the Decorated Function

greet("Pritam")

Now when you call greet("Pritam"), you’re actually calling wrapper_function("Pritam").

# Step 6: Output

The wrapper_function runs:

print("Before calling greet")

Calls the original greet("Pritam") → prints "Hello, Pritam!"

print("After calling greet")

# How it is working internally 

Our code:

@decorator_function
def greet(name):
    ...

Is transformed to:

greet = decorator_function(greet)

Which becomes:

greet = wrapper_function  # returned by decorator_function

Then:

greet("Pritam") → wrapper_function("Pritam")


# ❓ Question:

"Can you write a decorator that logs the execution time of any function it wraps?"

In [8]:
import time

def time_logger(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} executed in {end - start:.4f} seconds")
        return result
    return wrapper

# @time_logger
def slow_function():
    time.sleep(2)
    print("Done!")

slow_function()


Done!


In [10]:
import time

def time_logger(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} executed in {end - start:.4f} seconds")
        return result
    return wrapper

@time_logger
def slow_function():
    time.sleep(2)
    print("Done!")

slow_function()

Done!
slow_function executed in 2.0010 seconds


# Task: Write a decorator called authenticate that checks if a user is logged in (use a global user variable) before allowing access to a function.

In [11]:
# Global user object
user = {
    "username": "john_doe",
    "logged_in": False  # Toggle to True to simulate login
}

# Decorator
def authenticate(func):
    def wrapper(*args, **kwargs):
        if user.get("logged_in"):
            return func(*args, **kwargs)
        else:
            print("Access denied. User is not authenticated.")
    return wrapper

# Function protected by decorator
@authenticate
def view_dashboard():
    print("Welcome to your dashboard!")

# Test calls
print("Case 1: User not logged in")
view_dashboard()

print("\nCase 2: User logs in")
user["logged_in"] = True
view_dashboard()


Case 1: User not logged in
Access denied. User is not authenticated.

Case 2: User logs in
Welcome to your dashboard!


# Logging Decorator

Logs the function name and arguments every time it's called.

In [13]:
from functools import wraps

def log(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f"[LOG] Calling {func.__name__} with args: {args}, kwargs: {kwargs}")
        return func(*args, **kwargs)
    return wrapper

@log
def add(a, b):
    return a + b

print(add(10, 20))  # Logs the call
print(add(100, 32))  # Logs the call



[LOG] Calling add with args: (10, 20), kwargs: {}
30
[LOG] Calling add with args: (100, 32), kwargs: {}
132


# Timing Decorator

Measures how long a function takes to run (used in performance tuning).

In [15]:
import time
from functools import wraps

def timer(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"[TIMER] {func.__name__} ran in {end - start:.4f} seconds")
        return result
    return wrapper

@timer
def slow_operation():
    time.sleep(1)
    print("Done!")

slow_operation()

slow_operation()

slow_operation()


Done!
[TIMER] slow_operation ran in 1.0009 seconds
Done!
[TIMER] slow_operation ran in 1.0014 seconds
Done!
[TIMER] slow_operation ran in 1.0006 seconds


# Access Control Based on User Role

Allows function to run only for admin users.

In [ ]:
user = {
    "username": "john_doe",
    "role": "guest"
}

def require_role(role):
    def decorator(func):
        def wrapper(*args, **kwargs):
            if user.get("role") == role:
                return func(*args, **kwargs)
            else:
                print(f"Access denied. {user['role']} role cannot access this.")
        return wrapper
    return decorator

@require_role("admin")
def delete_account():
    print("Account deleted!")

delete_account()



Access denied. guest role cannot access this.
Access denied. guest role cannot access this.


# BUT

In [18]:
user = {
    "username": "Pritam",
    "role": "admin"
}

def require_role(role):
    def decorator(func):
        def wrapper(*args, **kwargs):
            if user.get("role") == role:
                return func(*args, **kwargs)
            else:
                print(f"Access denied. {user['role']} role cannot access this.")
        return wrapper
    return decorator

@require_role("admin")
def delete_account():
    print("Account deleted!")

delete_account()

Account deleted!


In [19]:
user = {
    "username": "Pritam",
    "role": "admin",
    "username": "Rahul",
    "role": "tester",

}

def require_role(role):
    def decorator(func):
        def wrapper(*args, **kwargs):
            if user.get("role") == role:
                return func(*args, **kwargs)
            else:
                print(f"Access denied. {user['role']} role cannot access this.")
        return wrapper
    return decorator

@require_role("admin")
def delete_account():
    print("Account deleted!")

delete_account()

Access denied. tester role cannot access this.


# Retry Decorator

Retries a function if it fails (useful in flaky network operations).

In [20]:
import random
from functools import wraps

def retry(n):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for attempt in range(1, n + 1):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    print(f"Attempt {attempt} failed: {e}")
            print("All retry attempts failed.")
        return wrapper
    return decorator

@retry(3)
def unstable_function():
    if random.random() < 0.7:
        raise Exception("Random failure")
    print("Success!")

unstable_function()


Success!


# # Cache Decorator (Memoization)

Stores previously computed values to save time.

In [21]:
def memoize(func):
    cache = {}
    def wrapper(n):
        if n not in cache:
            cache[n] = func(n)
        return cache[n]
    return wrapper

@memoize
def fib(n):
    if n <= 1:
        return n
    return fib(n - 1) + fib(n - 2)

print(fib(30))  # Much faster due to caching


832040


# Validate Arguments

Ensures arguments meet certain conditions before executing.

In [22]:
def validate_positive(func):
    def wrapper(x):
        if x <= 0:
            raise ValueError("Only positive numbers are allowed")
        return func(x)
    return wrapper

@validate_positive
def square_root(x):
    return x ** 0.5

print(square_root(16))   # Works
# print(square_root(-4)) # Raises ValueError


4.0


# Multiple Decorators on One Function

They stack — the closest to the function runs first.

In [23]:
def decorator1(func):
    def wrapper(*args, **kwargs):
        print("Decorator 1")
        return func(*args, **kwargs)
    return wrapper

def decorator2(func):
    def wrapper(*args, **kwargs):
        print("Decorator 2")
        return func(*args, **kwargs)
    return wrapper

@decorator1
@decorator2
def hello():
    print("Hello!")

hello()


Decorator 1
Decorator 2
Hello!


# Decorator to Throttle Function Calls

Goal: Prevent a function from being called more than once every X seconds.

In [24]:
import time
from functools import wraps

def throttle(seconds):
    def decorator(func):
        last_called = [0]  # Use a list to store mutable last call timestamp

        @wraps(func)
        def wrapper(*args, **kwargs):
            now = time.time()
            if now - last_called[0] < seconds:
                print(f"Throttled! Please wait {seconds - (now - last_called[0]):.2f} seconds.")
                return
            last_called[0] = now
            return func(*args, **kwargs)
        return wrapper
    return decorator

# Example usage
@throttle(5)  # Allow function call only once every 5 seconds
def greet():
    print("Hello, you called the function!")

# Testing the throttled function
if __name__ == "__main__":
    greet()      # Should work
    greet()      # Should be throttled
    time.sleep(5)
    greet()      # Should work again


Hello, you called the function!
Throttled! Please wait 5.00 seconds.
Hello, you called the function!


# Decorator to Count How Many Times a Function Has Been Called

Goal: Track how many times a function has been invoked during program execution.

In [25]:
from functools import wraps

# Dictionary to track function call counts
call_counts = {}

def count_calls(func):
    call_counts[func.__name__] = 0

    @wraps(func)
    def wrapper(*args, **kwargs):
        call_counts[func.__name__] += 1
        print(f"[INFO] {func.__name__} has been called {call_counts[func.__name__]} times.")
        return func(*args, **kwargs)
    return wrapper

# Example usage
@count_calls
def greet():
    print("Hello there!")

# Testing
if __name__ == "__main__":
    greet()
    greet()
    greet()

    print("\n[SUMMARY] Call counts:")
    print(call_counts)


[INFO] greet has been called 1 times.
Hello there!
[INFO] greet has been called 2 times.
Hello there!
[INFO] greet has been called 3 times.
Hello there!

[SUMMARY] Call counts:
{'greet': 3}


# Question:
Create three decorators:

@authenticate — allows function execution only if a global user is logged in.

@retry(n) — retries the decorated function up to n times if it raises an exception.

@log — logs the function name and arguments each time it is called.

Then, apply all three decorators to a function and demonstrate how they work together.

In [27]:
import random
from functools import wraps

# Global user state
user = {
    "username": "alice",
    "logged_in": True  # Change to False to test authentication block
}

# 1. Authenticate decorator
def authenticate(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        if user.get("logged_in"):
            return func(*args, **kwargs)
        else:
            print("Access denied. Please log in.")
    return wrapper

# 2. Retry decorator
def retry(n):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for attempt in range(1, n + 1):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    print(f"[Retry {attempt}/{n}] Exception: {e}")
            print("[Failure] All retry attempts failed.")
        return wrapper
    return decorator

# 3. Log decorator
def log(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f"[LOG] Calling {func.__name__} with args={args}, kwargs={kwargs}")
        return func(*args, **kwargs)
    return wrapper

# Example function using all decorators
@authenticate
@retry(3)
@log
def unstable_action():
    print("Attempting unstable action...")
    # Random failure simulation
    if random.random() < 0.7:
        raise Exception("Random failure!")
    print("Action succeeded!")

# Test the decorated function
if __name__ == "__main__":
    unstable_action()


[LOG] Calling unstable_action with args=(), kwargs={}
Attempting unstable action...
Action succeeded!


# Explanation

@authenticate: Checks if the user is logged in before running the function.

@retry(3): Retries the function up to 3 times if it throws an exception.

@log: Prints a log statement with the function name and arguments on every call.

The decorators are applied from the bottom up, so @log wraps unstable_action, then @retry wraps that, then @authenticate wraps the entire thing.